In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

# read our cleaned data into a pandas dataframe, specifying which variables are dates
oli_rfm = pd.read_csv('oli_all_for_rfm.csv', parse_dates = ['order_approved_at', 
                                                            'order_purchase_timestamp', 
                                                            'order_delivered_customer_date',
                                                            'order_estimated_delivery_date'])

print("oli_all lignes:", oli_rfm.shape[0],"colonnes:", oli_rfm.shape[1])


In [ ]:
oli_rfm = oli_rfm.drop('Unnamed: 0', axis = 1)

## Data Separation into 4 Different Time Periods

In [ ]:
# set date to beginning of 2017
before_time = dt.datetime.strptime("2016-12-30 23:59:00", "%Y-%m-%d %H:%M:%S")

# set date to end of april 2017
first_current_time = dt.datetime.strptime("2017-04-30 23:59:00", "%Y-%m-%d %H:%M:%S")

# set date to end of august 2017
second_current_time = dt.datetime.strptime("2017-08-31 23:59:00", "%Y-%m-%d %H:%M:%S")

# set date to end of december 2017
third_current_time = dt.datetime.strptime("2017-12-31 23:59:00", "%Y-%m-%d %H:%M:%S")

# set date to end of april 2018
fourth_current_time = dt.datetime.strptime("2018-04-30 23:59:00", "%Y-%m-%d %H:%M:%S")

# set date to end of august 2018
fifth_current_time = dt.datetime.strptime("2018-08-31 23:59:00", "%Y-%m-%d %H:%M:%S")


In [ ]:
# dataframe for period one : January 2017 to August 2017
olist_model_group_two = oli_rfm[(oli_rfm["order_purchase_timestamp"] > before_time) &
                                (oli_rfm["order_purchase_timestamp"] < second_current_time)].copy()

print("timeframe one data lignes:", olist_model_group_two.shape[0],"colonnes:", olist_model_group_two.shape[1])

# dataframe for period two:  January 2017 to December 2017
olist_model_group_three = oli_rfm[(oli_rfm["order_purchase_timestamp"] > before_time) &
                                (oli_rfm["order_purchase_timestamp"] < third_current_time)].copy()

print("timeframe two data lignes:", olist_model_group_three.shape[0],"colonnes:", 
      olist_model_group_three.shape[1])

# dataframe for period three: May 2017 to April 2018
olist_model_group_four = oli_rfm[(oli_rfm["order_purchase_timestamp"] > first_current_time) &
                                (oli_rfm["order_purchase_timestamp"] < fourth_current_time)].copy()

print("timeframe three data  lignes:", olist_model_group_four.shape[0],"colonnes:", 
      olist_model_group_four.shape[1])

# dataframe for period four: September 2017 to August 2018
olist_model_group_five = oli_rfm[(oli_rfm["order_purchase_timestamp"] > second_current_time) &
                                (oli_rfm["order_purchase_timestamp"] < fifth_current_time)].copy()

print("timeframe four data lignes:", olist_model_group_five.shape[0],"colonnes:", 
      olist_model_group_five.shape[1])


In [ ]:
def group_rfm_total_function(df, current_time):
    """ 
    Function to get the relevant variables for each time period, and also to aggregate that dataframe
    to the customer level.
    
    Parameters:
    df: period dataframe 
    current_time: end time of the dataframe period

    Returns:
    models_group: Returns the dataframe ready to be used for the models, agreggated at the customer level
    """
    
    df = get_rm_variables(df, "most_recent_purchase", "order_purchase_timestamp", 'max')
    
    # now that we have most recent purchase, we calculate the recency by subtracting the most recent purchase
    # to the "current date", so recency is days since last transaction
    df["Recency"] =  current_time - (df["most_recent_purchase"])
    
    df['Recency'] = df['Recency'].dt.days
    
    # get frequency
    nb_orders_per_cust = df.groupby("customer_unique_id").size()
    nb_orders_per_cust = pd.DataFrame(nb_orders_per_cust)
    nb_orders_per_cust.columns = ["Frequency"]
    df = pd.merge(left = df.copy(), right = nb_orders_per_cust.copy(), how = "outer", on = "customer_unique_id")

    # get monetary / total transaction value (over all orders of the customer)
    df = get_rm_variables(df, "Monetary", "total_payment_per_order", 'sum')
    
    # get average review 
    df = get_rm_variables(df, 'avg_review_score_per_customer', "review_score", 'mean')
    
    # get average delivery time
    df = get_round_rm_variables(df, 'avg_delivery_time_per_customer', 'delivery_time', 'mean')
    
    df = get_round_rm_variables(df, 'avg_difference_est_real_delivery_customer', 'difference_est_real_delivery',
                                'mean')
    
    # get average nb of items per client
    df = get_round_rm_variables(df, "avg_nb_of_items_per_customer", "nb_of_items_per_order",
                                'mean')
    
    # get average nb of payment installments per customer
    df = get_round_rm_variables(df, "avg_payment_installments_per_customer", 
                                "total_payment_installments_per_order",
                                'mean')

    # get frequent distance
    grouped = df.groupby("customer_unique_id")["distance_geopy"].agg(
    lambda x: pd.Series.mode(x)[0])

    grouped = pd.DataFrame(grouped)
    grouped.columns = ["geopy_frequnt_customerseller_distance"]

    df = pd.merge(left = df.copy(), right = grouped.copy(), how = "outer", on = "customer_unique_id")
    
    print("df original group lignes:", df.shape[0],"colonnes:", df.shape[1])

    models_group = df.drop_duplicates("customer_unique_id", keep = "first").copy()
    models_group = models_group.set_index('customer_unique_id')
    
    models_group = models_group[['Monetary', 'Recency', 'Frequency', 
                               'review_score', 
                               'avg_nb_of_items_per_customer', 
                               'avg_payment_installments_per_customer',
                               'delivery_time', 'difference_est_real_delivery',
                               "geopy_frequnt_customerseller_distance"]]

    print("new aggregated customer level lignes:", models_group.shape[0],"colonnes:", models_group.shape[1])
    return models_group


def get_rm_variables(df, new_column_name, column_for_groupby, transformation):
    """
    Function called by the group_rfm_total_function, to perform a groupby 
    
    Parameters:
    df: period time dataframe
    new_column_name: name for the column that will be created by the groupby transformation
    column_for_groupby: column that we wish to apply the transformation to
    transformation: the transformation to apply on the groupby 

    Returns:
    df: dataframe containing the original dataframe with the new column
    """
    df[new_column_name] = 0
    df[new_column_name] = df.groupby("customer_unique_id")[column_for_groupby].transform(transformation)
    return df


def get_round_rm_variables(df, new_column_name, column_for_groupby, transformation):
    """
    Function called by group_rfm_total_function, to perform a groupby and the round the result at the end.

    Parameters:
    df: period time dataframe
    new_column_name: name for the column that will be created by the groupby transformation
    column_for_groupby: column that we wish to apply the transformation to
    transformation: the transformation to apply on the groupby 

    Returns:
    df: dataframe containing the original dataframe with the new column
    """
    
    df[new_column_name] = 0
    df[new_column_name] = round(df.groupby("customer_unique_id")[column_for_groupby].transform(transformation))
    
    return df

In [ ]:
models_group_one  = group_rfm_total_function(olist_model_group_two, second_current_time)

In [ ]:
models_group_two  = group_rfm_total_function(olist_model_group_three, third_current_time)

In [ ]:
models_group_three  = group_rfm_total_function(olist_model_group_four, fourth_current_time)

In [ ]:
models_group_four  = group_rfm_total_function(olist_model_group_five, fifth_current_time)

# Data for Timeframe 1

## RFM

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# take just the three RFM variables from the dataframe for period one
rfm_group_one = models_group_one[['Monetary', 'Recency', 'Frequency']]

In [ ]:
rfm_group_one.Recency.describe()

In [ ]:
rfm_group_one.Frequency.describe()

In [ ]:
rfm_group_one.Monetary.describe()

In [ ]:
sns.histplot(rfm_group_one.Frequency)

In [ ]:
rfm_group_one['Frequency'].value_counts()

In [ ]:
# create quantiles for the variables in the dataframe
rfm_quantiles = rfm_group_one.quantile(q = [0.25, 0.5, 0.75])
rfm_quantiles["Frequency"][0.25] = 2
rfm_quantiles["Frequency"][0.5] = 3
rfm_quantiles["Frequency"][0.75] = 4

# send quantile to dictionary
rfm_quantiles = rfm_quantiles.to_dict()

In [ ]:
def Rmf_score_function(x, p, rfm_quantiles):
    """
    Function to get a score based on the quantiles. 1 is highest, best, and 4 is lowest, worst. 
    Parameters:
    x = value in question that is being compared to the quantile value
    p = the variable, either : Monetary, Frequency
    rfm_quantiles = quartiles dict

    Returns:
    Returns the score for each value
    """
    if x >= rfm_quantiles[p][0.75]:
        return 1
    elif x >= rfm_quantiles[p][0.50]:
        return 2
    elif x >= rfm_quantiles[p][0.25]:
        return 3
    else:
        return 4
    

def Rmf_score_recency(x, p, rfm_quantiles): 
    """
    Function to get a score based on the quantiles. 
    The way the scoring is put in place is reversed compared to the previous function. 
    1 is still best. 
   
    Parameters:
    x = value in question that is being compared to the quantile value
    p = the variable, either : Monetary, Frequency
    rfm_quantiles = quartiles dict

    Returns:
    Returns the score for each value
    """
    if x <= rfm_quantiles[p][0.25]:
        return 1
    elif x <= rfm_quantiles[p][0.50]:
        return 2
    elif x <= rfm_quantiles[p][0.75]:
        return 3
    else:
        return 4

In [ ]:
rfm_group_one['R_score'] = rfm_group_one['Recency'].apply(Rmf_score_recency, args = ('Recency', 
                                                                                     rfm_quantiles))
rfm_group_one['F_score'] = rfm_group_one['Frequency'].apply(Rmf_score_function, args = ('Frequency', 
                                                                                        rfm_quantiles))
rfm_group_one['M_score'] = rfm_group_one['Monetary'].apply(Rmf_score_function, args = ('Monetary', 
                                                                                       rfm_quantiles))

rfm_group_one['RFM_total_class'] = rfm_group_one.R_score.map(str) + rfm_group_one.F_score.map(str) \
                                       + rfm_group_one.M_score.map(str)

In [ ]:
def rfm_get_label(df):
    """
    Function to get labels for each data point according to the RFM scores.

    Parameters:
    df: dataframe containing the scores for each data point

    Returns:
    The labels for each data points.
    """
    
    if ((df['R_score'] == 1) and (df['F_score'] <= 2) and (df['M_score'] <= 2)):
        return 'Top Customers'
    
    elif ((df['R_score'] == 1) and (df['F_score'] <= 3)):
        return 'Active Customers'
    
    elif ((df['R_score'] == 2) and (df['F_score'] <= 3) and (df['M_score'] <=2)):
        return 'Emerging Customers'
    
    elif ((df['R_score'] <= 2)):
        return 'Unsteady Customers'
    
    
    elif ((df['R_score'] == 4) and (df['F_score']) >= 3 and (df['M_score'] >= 3)):
        return 'Lost Customers'
    
    elif ((df['R_score'] == 4) and (df['F_score'] >= 3) and (df['M_score'] <=2)):
        return 'Inactive Customers'
    
    elif ((df['R_score'] == 3) and (df['F_score'] >= 3) and (df['M_score'] >=3)):
        return 'Potentially Lost Customers'

    else:
        return 'At Risk Customers'


In [ ]:
# Top Customers  
# Active Customers (recent but usual don't combine high monetary and high frequency like the top customers)
# Emerging Customers (a little less recent than active customers)
# Unsteady Customers (quite recent but didn't get into other 'good' categories, plus people who are recent but
# not frequent at all)

# At Risk Customers (no so recent anymore)
# Potentially Lost Customers (not recent and not too frequent either)
# Inactive Customers (long time ago, not recent at all, but high monetary)
# Lost Customers (long time ago, not frequent, low monetary)

rfm_group_one['rfm_label'] = rfm_group_one.apply(rfm_get_label, axis = 1)

rfm_group_one['rfm_label'].value_counts()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Table(
    header=dict(values= ['Labels', 'Number of Customers'],
                line_color='darkslategray',
                fill_color='lightskyblue',
                align='left'),
    cells=dict(values=[['Unsteady Customers', 'At Risk Customers', 'Inactive Customers', 'Lost Customers',
                        'Potentially Lost Customers', 'Active Customers', 'Emerging Customers', 'Top Customers'],
                       [10405, 2761, 2662, 2635, 2626, 157, 124, 19]], 
               line_color='darkslategray',
               fill_color='lightcyan',
               align='left'))
])

fig.update_layout(width=500, height= 600)
fig.show()

In [ ]:
import plotly.express as px

# plot the clusters vs their recency
df = px.data.tips()
fig = px.box(rfm_group_one, x ='rfm_label', y = 'Recency', color = "rfm_label")
fig.update_layout(showlegend = False, font_size = 14, width = 700, height = 600, 
                  yaxis_title = "Recency / Most Recent Order per Customer")
fig.show()

In [ ]:
fig = px.box(rfm_group_one, x ='rfm_label', y = 'Monetary', color = "rfm_label")
fig.update_layout(showlegend = False, font_size = 14, width = 700, height = 600,
                 yaxis_title = "Monetary / Total Spent per Customer")
fig.show()

In [ ]:
fig = px.box(rfm_group_one, x ='rfm_label', y = 'Frequency', color = "rfm_label")
fig.update_layout(showlegend = False, font_size = 14, width = 700, height = 600)
fig.show()

## Data for Timeframe 1 K-Means


In [ ]:
X_period_one = models_group_one[['Monetary', 'Recency', 'Frequency', 
                                 'review_score',
                                 'avg_payment_installments_per_customer', 
                                 'delivery_time']]

In [ ]:
import plotly.graph_objects as go

# plot the boxplots without the monetary variable

y2 = models_group_one["Frequency"]
y3 = models_group_one["review_score"]
y4 = models_group_one["avg_payment_installments_per_customer"]


fig = go.Figure()
fig.add_trace(go.Box(y = y2, name = 'Frequency per Customer',
                marker_color = 'burlywood'))
fig.add_trace(go.Box(y = y3, name = 'Average Review Score per Customer',
                marker_color = 'royalblue'))
fig.add_trace(go.Box(y = y4, name = 'Average Payment Instalments per Customer',
                marker_color = 'rebeccapurple'))

fig.update_layout(showlegend = False, font_size = 14,
                 width = 900, height = 650)
fig.show()

In [ ]:
import plotly.express as px
df = models_group_one
fig = px.histogram(df, x = "Recency",
                   marginal = "box")
fig.update_layout(showlegend = False, font_size = 14, xaxis_title = "Recency / Most Recent Order per Customer",
                 width = 700, height = 600)
fig.show()

In [ ]:
df = models_group_one
fig = px.histogram(df, x = "Monetary",
                   marginal = "box")
fig.update_layout(showlegend = False, font_size = 14, xaxis_title = "Monetary / Total Spent per Customer",
                 width = 700, height = 600)
fig.show()

In [ ]:
df = models_group_one
fig = px.histogram(df, x = 'delivery_time',
                   marginal = "box")
fig.update_layout(showlegend = False, font_size = 14, xaxis_title = "Average Delivery Time per Customer",
                 width = 700, height = 600)
fig.show()

In [ ]:
X_period_one.quantile(q = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [ ]:
def preproc_freq(df):
    """
    Function to pre-process the Frequency data points, before using the Power Transform

    Parameters:
    df: dataframe for a given time period

    Returns:
    df: returns the dataframe, with Frequency, pre-processed before the power transform
    """
    
    quants = df.quantile(q = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
    
    quants["Frequency"][0.4] = 2
    quants["Frequency"][0.5] = 3
    quants["Frequency"][0.6] = 4
    quants["Frequency"][0.7] = 5
    quants["Frequency"][0.8] = 6
    quants["Frequency"][0.9] = 7
    
    #send quantile to dictionary
    quants = quants.to_dict()
    
    df['F_score'] = df['Frequency'].apply(preproc_score_function, args = ('Frequency', quants))

    df['M_score'] = df['Monetary']
   
    df['R_score'] = df['Recency']
    
    df['rev_score'] = df['review_score']
    
    df['pay_inst_score'] = df['avg_payment_installments_per_customer']
        
    df['del_time'] = df['delivery_time']
    
    return df
   

In [ ]:
def preproc_score_function(x, p, rfm_quantiles):
    """
    Function called by preproc_freq
    
    # where 10 is higher (best) 1 is lower (bad)
    # the lower the frequency, the worse the score
    # Arguments (x = value, p = Frequency, rfm_quantiles = quartiles dict)

    Parameters:
    x = value in question that is being compared to the quantile value
    p = the variable
    rfm_quantiles = quartiles dict

    Returns:
    Returns a score from 1 to 10
    """
        
    if x <= rfm_quantiles[p][0.1]:
        return 1
    elif x <= rfm_quantiles[p][0.2]:
        return 2
    elif x <= rfm_quantiles[p][0.3]:
        return 3
    elif x <= rfm_quantiles[p][0.4]:
        return 4
    elif x <= rfm_quantiles[p][0.5]:
        return 5
    elif x <= rfm_quantiles[p][0.6]:
        return 6
    elif x <= rfm_quantiles[p][0.7]:
        return 7
    elif x <= rfm_quantiles[p][0.8]:
        return 8
    elif x <= rfm_quantiles[p][0.9]:
        return 9
    else:
        return 10

In [ ]:
X_rfm_scaled = preproc_freq(X_period_one)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer

def my_pre_process(df):
    """
    Function which applies the Power Transform to all the variables.
    
    Parameters:
    df: the dataframe containing the variables to be processed

    Returns:
    X_rfm_scaled: dataframe containing variables after the power transform has been applied
    """
    
    #scaler = StandardScaler()
    #qt = QuantileTransformer(n_quantiles = 10, random_state=0)
    
    pt = PowerTransformer()

    X_rfm_scaled = pt.fit_transform(df.copy())

    X_rfm_scaled = pd.DataFrame(X_rfm_scaled, index = df.index, columns = df.columns)
    
    return X_rfm_scaled

In [ ]:
X_rfm_scaled = my_pre_process(X_rfm_scaled[['M_score', 'R_score', 'F_score',
                                            "rev_score", 'pay_inst_score','del_time']].copy())
  
X_rfm_scaled.columns = ['Monetary', 'Recency', 'Frequency', 'review_score', 
                        'avg_payment_installments_per_customer', 'delivery_time']


In [ ]:
import plotly.graph_objects as go

y1 = X_rfm_scaled["Monetary"]
y2 = X_rfm_scaled["Recency"]
y3 = X_rfm_scaled["Frequency"]

y4 = X_rfm_scaled["review_score"]
y5 = X_rfm_scaled["avg_payment_installments_per_customer"]
y6 = X_rfm_scaled['delivery_time']
 
fig = go.Figure()
fig.add_trace(go.Box(y = y1, name = 'Monetary / Total Spent per Customer',
                marker_color = 'indianred'))
fig.add_trace(go.Box(y = y2, name = 'Recency / Most Recent Order per Customer',
                marker_color = 'lightseagreen'))
fig.add_trace(go.Box(y = y3, name = 'Frequency per Customer',
                marker_color = 'burlywood'))

fig.add_trace(go.Box(y = y4, name = 'Avg Review Score',
                marker_color = 'royalblue'))
fig.add_trace(go.Box(y = y5, name = 'Avg Payment Instalments per Customer',
                marker_color = 'rebeccapurple'))
fig.add_trace(go.Box(y = y6, name = 'Avg Delivery Time',
                marker_color = "goldenrod"))

fig.update_layout(showlegend = False, font_size = 14,
             width = 700, height = 600)
fig.show()

In [ ]:
import plotly.express as px
df = X_rfm_scaled
fig = px.histogram(df, x = "delivery_time",
                   marginal = "box")
fig.update_layout(showlegend = False, font_size = 14, xaxis_title = "Avg Delivery Time per Customer",
                 width = 700, height = 600)
fig.show()

Visualisations and Decisions for Number of Clusters

In [ ]:
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from sklearn import cluster, metrics
import matplotlib

matplotlib.rcParams["figure.figsize"] = (9,6)

model = KMeans()
visualizer = KElbowVisualizer(model, k = (2,12))

visualizer.fit(X_rfm_scaled)    # Fit the data to the visualizer
visualizer.poof() 

In [ ]:
# Instantiate the clustering model and visualizer 
model = KMeans()
visualizer = KElbowVisualizer(model, k = (4,12), metric = 'calinski_harabasz', timings = False)

visualizer.fit(X_rfm_scaled)    # Fit the data to the visualizer
visualizer.poof() 

In [ ]:
# Instantiate the clustering model and visualizer 
model = KMeans(4)
visualizer = SilhouetteVisualizer(model)

visualizer.fit(X_rfm_scaled)    # Fit the data to the visualizer
visualizer.poof()    # Draw/show/poof the data

In [ ]:
# Instantiate the clustering model and visualizer 
model = KMeans(5)
visualizer = SilhouetteVisualizer(model)

visualizer.fit(X_rfm_scaled)    # Fit the data to the visualizer
visualizer.poof()    # Draw/show/poof the data

In [ ]:
%pylab inline
from sklearn import cluster, metrics

silhouettes = []
for num_clusters in range(3,10):
    cls = cluster.KMeans(n_clusters = num_clusters)
                         #n_init = 50, init = 'random')
    cls.fit(X_rfm_scaled)
    silh = metrics.silhouette_score(X_rfm_scaled, cls.labels_)
    silhouettes.append(silh)
    
plt.plot(range(3, 10), silhouettes, marker = 'o')

## changes with n_init, I first got 2 as max and now it's 5

In [ ]:
import plotly.graph_objects as go

x = np.arange(3,10)

fig = go.Figure(data = go.Scatter(x = x, y = silhouettes))
fig.update_layout(showlegend = False, font_size = 14, xaxis_title = "Number of Clusters",
                  yaxis_title = "Silhouette Scores",
                  width = 700, height = 600)
fig.show()

## Data for Timeframe 1: KMeans with Determined Nb of Clusters

In [ ]:
%pylab inline
from sklearn import cluster, metrics

# we choose 5 clusters for the k means
cls = cluster.KMeans(n_clusters = 5)
cls.fit(X_rfm_scaled)
cls.labels_

In [ ]:
# initialize a dataframe that will contain the silhouette scores for the different time periods
kmeans_silh = []
kmeans_silh.append(metrics.silhouette_score(X_rfm_scaled, cls.labels_))

In [ ]:
group1_for_reunion = pd.concat([X_rfm_scaled.copy(), pd.Series(cls.labels_, index = X_rfm_scaled.index)], axis = 1)
group1_for_reunion.columns = ['Monetary', 'Recency',
                              'Frequency', 
                              'review_score', 
                              'avg_payment_installments_per_customer', 
                              'delivery_time',
                              'cluster']

group1_for_reunion["cluster"].value_counts()

In [ ]:
for_sankey_period_one = pd.DataFrame(group1_for_reunion["cluster"])

Boxplot Visualisation

In [ ]:
boxplot_df_period_one = pd.concat([X_period_one.copy(), group1_for_reunion["cluster"].copy()], axis = 1)

In [ ]:
import plotly.express as px

def boxplot_individ_variable_visual(df, y_vari, yaxis_title):
    """
    Function to get boxplots illustrating the clusters vs different variables
    Parameters:
    df: dataframe containing the variables and clusters
    y_vari: variable for the y axis
    yaxis_title: title of the y axis

    Returns:
    Returns the figure
    """
    fig = px.box(df, x = "cluster", y = y_vari, color = "cluster")
    
    fig.update_layout(showlegend = False, font_size = 14, 
                      yaxis_title = yaxis_title, xaxis_title = "Cluster",
                      width = 700, height = 600)
    
    fig.show()

In [ ]:
boxplot_individ_variable_visual(group1_for_reunion, "Monetary", "Monetary / Total Spent per Customer")

In [ ]:
boxplot_individ_variable_visual(group1_for_reunion, "Frequency", "Frequency per Customer")

In [ ]:
boxplot_individ_variable_visual(group1_for_reunion, "Recency", "Recency / Most Recent Order per Customer")

In [ ]:
boxplot_individ_variable_visual(group1_for_reunion, 'avg_payment_installments_per_customer',
                               "Avg Payments Instalments per Customer")

In [ ]:
boxplot_individ_variable_visual(group1_for_reunion, 'review_score', "Avg Review Score per Customer")

In [ ]:
boxplot_individ_variable_visual(group1_for_reunion, 'delivery_time', "Avg Delivery Time per Customer")

Plot Clusters with PCA

In [ ]:
from sklearn import decomposition

# use the 3 first PCA components
pca = decomposition.PCA(n_components = 3)
pca.fit(X_rfm_scaled)
print(pca.explained_variance_ratio_.cumsum())
X_trans = pca.transform(X_rfm_scaled)

In [ ]:
# print the explained variance ratio of each axis
pca.explained_variance_ratio_

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

cls_graph = cluster.KMeans(n_clusters = 5)
cls_graph.fit(X_rfm_scaled)

fig = px.scatter(x = X_trans[:,0], y = X_trans[:,1], color = cls_graph.labels_)

fig.update_layout(height = 600, width = 600, xaxis_title = "F1 (24%)", yaxis_title = "F2 (20%)")
fig.show()



In [ ]:
import plotly.express as px

fig = px.scatter_3d(x = X_trans[:,0], y = X_trans[:,1], z = X_trans[:,2],
              color = cls_graph.labels_, opacity = 0.45)
fig.update_layout(height = 600, width = 600,
                 xaxis_title = "F1 (24%)", yaxis_title = "F2 (20%)")
                  #, zaxis_title = "F3 (16%)")

fig.show()

## Data for Timeframe 1 : DBScan

In [ ]:
from sklearn.cluster import DBSCAN

# DBScan on data for the first timeframe
db_clust = DBSCAN(min_samples = 150, eps = 0.95).fit(X_rfm_scaled)
db_clust.labels_


In [ ]:
db_period_one = pd.concat([X_rfm_scaled.copy(), pd.Series(db_clust.labels_,
                                                          index = X_rfm_scaled.index)], axis = 1)
db_period_one.columns = ['Monetary', 'Recency',
                         'Frequency', 
                         'review_score', 
                         'avg_payment_installments_per_customer', 
                         'delivery_time',
                         'cluster']

In [ ]:
db_period_one["cluster"].value_counts()

In [ ]:
boxplot_individ_variable_visual(db_period_one, "Monetary", "Monetary / Total Spent per Customer")

In [ ]:
boxplot_individ_variable_visual(db_period_one, "Recency", "Recency / Most Recent Order per Customer")

In [ ]:
boxplot_individ_variable_visual(db_period_one, "Frequency", "Frequency per Customer")

In [ ]:
boxplot_individ_variable_visual(db_period_one, 'review_score', "Avg Review Score per Customer")

In [ ]:
boxplot_individ_variable_visual(db_period_one, 'avg_payment_installments_per_customer',
                               "Avg Payments Instalments per Customer")

# Data for Timeframe 2

In [ ]:
X_period_two = models_group_two[['Monetary', 'Recency', 'Frequency', "review_score", 
                                 'avg_payment_installments_per_customer', 
                                 'delivery_time']]

In [ ]:
# preprocessing on the data for the second timeframe
X_rfm_scaled2 = preproc_freq(X_period_two)

In [ ]:
X_rfm_scaled2 = my_pre_process(X_rfm_scaled2[['M_score', 'R_score', 'F_score', 
                                              'rev_score', 
                                              'pay_inst_score', 
                                              'del_time']])
    
X_rfm_scaled2.columns = ['Monetary', 'Recency', 'Frequency', 
                         'review_score', 
                         'avg_payment_installments_per_customer', 
                         'delivery_time']

In [ ]:
matplotlib.rcParams["figure.figsize"] = (9,6)

model = KMeans()
visualizer = KElbowVisualizer(model, k = (2,12))

visualizer.fit(X_rfm_scaled2)    # Fit the data to the visualizer
visualizer.poof() 

## Data for Timeframe 2: Kmeans

In [ ]:
def get_sankey_dfs(scaled_df, labels):
    """
    Function to get dataframes that will be used to produce a sankey diagram

    Parameters:
    scaled_df: processed data
    labels: cluster labels

    Returns:
    for_newfit_sankey: dataframe containing just the cluster labels
    group_new_fit: dataframe containing the processed data and it's corresponding cluster label
    """
    
    group_new_fit = pd.concat([scaled_df.copy(), pd.Series(labels, index = scaled_df.index)], axis = 1)
    group_new_fit.columns = ['Monetary', 'Recency', 'Frequency', "review_score", 
                             "avg_payment_installments_per_customer", 'delivery_time',
                             'cluster']
    for_newfit_sankey = pd.DataFrame(group_new_fit["cluster"])
    
    return for_newfit_sankey, group_new_fit

In [ ]:
%pylab inline
from sklearn import cluster, metrics

# create a new kmeans model fitted to the second timeframe data
cls2 = cluster.KMeans(n_clusters = 5)
cls2.fit(X_rfm_scaled2)

In [ ]:
# append the dataframe that records the silhouette scores for the kmeans models
kmeans_silh.append(metrics.silhouette_score(X_rfm_scaled2, cls2.labels_))

In [ ]:
kmeans_silh

In [ ]:
# apply the get_sankey_dfs fuction to our second timeframe data
for_newfit_sankey2, group_new_fit2 = get_sankey_dfs(X_rfm_scaled2, cls2.labels_)
group_new_fit2["cluster"].value_counts()

In [ ]:
boxplot_individ_variable_visual(group_new_fit2, 'avg_payment_installments_per_customer',
                               "Avg Payments Instalments per Customer")

In [ ]:
boxplot_individ_variable_visual(group_new_fit2, "Frequency", "Frequency per Customer")

In [ ]:
boxplot_individ_variable_visual(group_new_fit2, "Recency", "Recency / Most Recent Order per Customer")

Use Model from 1st Period on 2nd Period

In [ ]:
predict2 = cls.predict(X_rfm_scaled2)

In [ ]:
for_previous_fitsankey2, group_nprev_fit2 = get_sankey_dfs(X_rfm_scaled2, predict2)

Compute Rand Index Adjusted for chance

In [ ]:
for_newfit_sankey2["cluster"]

In [ ]:
from sklearn.metrics.cluster import adjusted_rand_score

# initialize dataframe to keep track of ARI scores
kmeans_ari = []
kmeans_ari.append(adjusted_rand_score(for_previous_fitsankey2["cluster"], for_newfit_sankey2["cluster"]))

In [ ]:
kmeans_ari

## Data for Timeframe 2 : DBScan

In [ ]:
from sklearn.cluster import DBSCAN

db_clust_period_two = DBSCAN(min_samples = 150, eps = 0.80).fit(X_rfm_scaled2)
db_clust_period_two.labels_

In [ ]:
db_period_two = pd.concat([X_rfm_scaled2.copy(), pd.Series(db_clust_period_two.labels_,
                                                           index = X_rfm_scaled2.index)], axis = 1)
db_period_two.columns = ['Monetary', 
                         'Recency',
                         'Frequency', 
                         'review_score', 
                         'avg_payment_installments_per_customer', 
                         'delivery_time',
                         'cluster']

db_period_two["cluster"].value_counts()

In [ ]:
boxplot_individ_variable_visual(db_period_two, "Recency")

In [ ]:
boxplot_individ_variable_visual(db_period_two, "Monetary")

In [ ]:
boxplot_individ_variable_visual(db_period_two, "Frequency")

# Data for Timeframe 3

In [ ]:
X_period_three = models_group_three[['Monetary', 'Recency', 'Frequency',
                                     'review_score', 
                                     'avg_payment_installments_per_customer',
                                     'delivery_time']]

In [ ]:
X_rfm_scaled3 = preproc_freq(X_period_three)

In [ ]:
X_rfm_scaled3 = my_pre_process(X_rfm_scaled3[['M_score', 'R_score', 'F_score', 
                                              'rev_score', 'pay_inst_score', 'del_time']])

X_rfm_scaled3.columns = ['Monetary', 'Recency', 'Frequency', "review_score", 
                         'avg_payment_installments_per_customer',
                         'delivery_time']


In [ ]:
matplotlib.rcParams["figure.figsize"] = (9,6)

model = KMeans()
visualizer = KElbowVisualizer(model, k = (2,12))

visualizer.fit(X_rfm_scaled3)    # Fit the data to the visualizer
visualizer.poof() 

## Data for Timeframe 3: Kmeans

In [ ]:
%pylab inline
from sklearn import cluster, metrics

# kmeans fitted to the third time period
cls3 = cluster.KMeans(n_clusters = 5)
cls3.fit(X_rfm_scaled3)

In [ ]:
kmeans_silh.append(metrics.silhouette_score(X_rfm_scaled3, cls3.labels_))

In [ ]:
# apply the get_sankey_dfs function to this timeframe for the sankey diagram later
for_newfit_sankey3, group_new_fit3 = get_sankey_dfs(X_rfm_scaled3, cls3.labels_)
group_new_fit3["cluster"].value_counts()

In [ ]:
boxplot_individ_variable_visual(group_new_fit3,'avg_payment_installments_per_customer',
                               "Avg Payments Instalments per Customer")

In [ ]:
boxplot_individ_variable_visual(group_new_fit3,  "Frequency", "Frequency per Customer")

In [ ]:
boxplot_individ_variable_visual(group_new_fit3, 'review_score', "Avg Review Score per Customer")

In [ ]:
boxplot_individ_variable_visual(group_new_fit3, "Recency", "Recency / Most Recent Order per Customer")

Use Model from 1st Period on 2nd Period

In [ ]:
predict3 = cls.predict(X_rfm_scaled3)

In [ ]:
for_previous_fitsankey3, group_prev_fit3  = get_sankey_dfs(X_rfm_scaled3, predict3)


In [ ]:
from sklearn.metrics.cluster import adjusted_rand_score

kmeans_ari.append(adjusted_rand_score(for_previous_fitsankey3["cluster"], for_newfit_sankey3["cluster"]))

In [ ]:
kmeans_ari

## Data for Timeframe 3: DBScan 

In [ ]:
from sklearn.cluster import DBSCAN

db_clust_period_three = DBSCAN(min_samples = 150, eps = 0.90).fit(X_rfm_scaled3)
db_clust_period_three.labels_

In [ ]:
db_period_three = pd.concat([X_rfm_scaled3.copy(), pd.Series(db_clust_period_three.labels_,
                                                          index = X_rfm_scaled3.index)], axis = 1)
db_period_three.columns = ['Monetary', 'Recency',
                           'Frequency', 
                           'review_score', 
                           'avg_payment_installments_per_customer', 
                           'delivery_time',
                           'cluster']

db_period_three["cluster"].value_counts()

In [ ]:
boxplot_individ_variable_visual(db_period_three, "Recency")

# Data for Timeframe 4

In [ ]:
X_period_four = models_group_four[['Monetary', 'Recency', 'Frequency', 
                                   'review_score', 
                                   'avg_payment_installments_per_customer', 
                                   'delivery_time']]

In [ ]:
X_rfm_scaled4 = preproc_freq(X_period_four)

In [ ]:
X_rfm_scaled4 = my_pre_process(X_rfm_scaled4[['M_score', 'R_score', 'F_score', 
                                              'rev_score', 
                                              'pay_inst_score', 
                                              'del_time']])
    
X_rfm_scaled4.columns = ['Monetary', 'Recency', 'Frequency', 
                         'review_score', 
                         'avg_payment_installments_per_customer', 'delivery_time']

In [ ]:
matplotlib.rcParams["figure.figsize"] = (9,6)

model = KMeans()
visualizer = KElbowVisualizer(model, k = (2,12))

visualizer.fit(X_rfm_scaled4)    # Fit the data to the visualizer
visualizer.poof() 

## Data for Timeframe 4: Kmeans

In [ ]:
cls4 = cluster.KMeans(n_clusters = 5)
cls4.fit(X_rfm_scaled4)

In [ ]:
kmeans_silh.append(metrics.silhouette_score(X_rfm_scaled4, cls4.labels_))

In [ ]:
for_newfit_sankey4, group_new_fit4 = get_sankey_dfs(X_rfm_scaled4, cls4.labels_)
group_new_fit4["cluster"].value_counts()

In [ ]:
boxplot_individ_variable_visual(group_new_fit4, 'review_score', "Avg Review Score per Customer")

In [ ]:
boxplot_individ_variable_visual(group_new_fit4, "Frequency", "Frequency per Customer")

In [ ]:
boxplot_individ_variable_visual(group_new_fit4, "Recency", "Recency / Most Recent Order per Customer")

Use Model from 1st Period on 2nd Period

In [ ]:
predict4 = cls.predict(X_rfm_scaled4)

In [ ]:
for_previous_fitsankey4, group_prev_fit4 = get_sankey_dfs(X_rfm_scaled4, predict4)

In [ ]:
from sklearn.metrics.cluster import adjusted_rand_score

kmeans_ari.append(adjusted_rand_score(for_previous_fitsankey4["cluster"], for_newfit_sankey4["cluster"]))

In [ ]:
kmeans_ari

In [ ]:
kmeans_silh

## Data for Timeframe 4 : DBScan 

In [ ]:
from sklearn.cluster import DBSCAN

db_clust_period_four = DBSCAN(min_samples = 150, eps = 0.90).fit(X_rfm_scaled4)
db_clust_period_four.labels_

In [ ]:
db_period_four = pd.concat([X_rfm_scaled4.copy(), pd.Series(db_clust_period_four.labels_,
                                                          index = X_rfm_scaled4.index)], axis = 1)
db_period_four.columns = ['Monetary', 'Recency',
                           'Frequency', 
                           'review_score', 
                           'avg_payment_installments_per_customer', 
                           'delivery_time',
                           'cluster']

db_period_four["cluster"].value_counts()

In [ ]:
boxplot_individ_variable_visual(db_period_four, "Recency")

# All Time Frame Period Groups 

In [ ]:
def make_sankey_df():
    """
    Function to get a dataframe for the sankey diagram with labels and colours of the flows and clusters

    Parameters:

    Returns:
    sankey_chart: Returns dataframe containing the colours and labels
    """
    
    sankey_chart = pd.DataFrame()

    sankey_chart["colour"] = ["lightcoral", "gold", "plum", "deepskyblue", "bisque",
                              "lightcoral", "gold", "plum", "deepskyblue", "bisque",
                              "lightcoral", "gold", "plum", "deepskyblue", "bisque",
                              "lightcoral", "gold", "plum", "deepskyblue", "bisque",]
        
    sankey_chart["label"] = ["Unhappy Clients", "Active", "Best Clients", "Multiple Instalments Clients", "Inactive",
                             "Unhappy Clients", "Active", "Best Clients", "Multiple Instalments Clients", "Inactive",
                             "Unhappy Clients", "Active", "Best Clients", "Multiple Instalments Clients", "Inactive",
                             "Unhappy Clients", "Active", "Best Clients", "Multiple Instalments Clients", "Inactive"]
    return sankey_chart


In [ ]:
def make_sankey_newfit_df():
    """
    Function to get a dataframe for the sankey diagram with labels and colours of the flows and clusters

    Parameters:

    Returns:
    sankey_chart: Returns dataframe containing the colours and labels
    """
    
    sankey_chart = pd.DataFrame()

    sankey_chart["colour"] = ["lightcoral", "gold", "plum", "deepskyblue", "bisque",
                              "bisque", "deepskyblue", "gold", "lightcoral", "plum",
                              "bisque", "deepskyblue", "gold", "plum",  "lightcoral",
                              "bisque", "deepskyblue", "lightcoral", "gold", "plum"]
        
    sankey_chart["label"] = ["Unhappy Clients", "Active", "Best Clients", "Multiple Instalments Clients", "Inactive",
                             "Inactive", "Multiple Instalments Clients", "Active", "Unhappy Clients", "Best Clients",
                             "Inactive", "Multiple Instalments Clients", "Active", "Best Clients", "Unhappy Clients",
                             "Inactive", "Multiple Instalments Clients", "Unhappy Clients", "Active", "Best Clients"]
    return sankey_chart


In [ ]:
sankey_chart_previous_fit = make_sankey_df()

sankey_chart_newfit = make_sankey_newfit_df()

In [ ]:
def make_df_for_sankey(sankey_chart, nb_of_clust, for_sankey1, for_sankey2, for_sankey3, for_sankey4):
    """
    Function that produces a dataframe that will build the sankey diagram

    Parameters:
    sankey_chart: dataframe resulting from our function make_sankey_df
    nb_of_clust: number of clusters used in the kmeans
    for_sankey1: dataframe with the info for timeframe 1
    for_sankey2: dataframe with the info for timeframe 2
    for_sankey3: dataframe with the info for timeframe 3
    for_sankey4: dataframe with the info for timeframe 4

    Returns:
    sankey_chart: dataframe containing the information to build the sankey diagram
    """
    
    sankey_chart["value"] = np.nan
    sankey_chart["source"] = np.nan
    sankey_chart["target"] = np.nan
    
    for i in range(0, nb_of_clust):
        for_sankey2[for_sankey2["cluster"] == i] = i + nb_of_clust

    for i in range(0, nb_of_clust):
        for_sankey3[for_sankey3["cluster"] == i] = i + (nb_of_clust * 2)
        
    for i in range(0, nb_of_clust):
        for_sankey4[for_sankey4["cluster"] == i] = i + (nb_of_clust * 3)
        
    index = 0

    for i in range(0, nb_of_clust):
        for j in range(nb_of_clust, (nb_of_clust*2)):
            
            merge = pd.merge(left = for_sankey1[for_sankey1["cluster"] == i].copy(), 
                             right = for_sankey2[for_sankey2["cluster"] == j].copy(),
                             how = "inner", on = "customer_unique_id")
            
            sankey_chart.loc[index,"value"] = merge.shape[0]
            sankey_chart.loc[index,"source"] = i
            sankey_chart.loc[index,"target"] = j
            index += 1
            
            merge = pd.merge(left = for_sankey2[for_sankey2["cluster"] == (i+nb_of_clust)].copy(), 
                             right = for_sankey3[for_sankey3["cluster"] == (j+nb_of_clust)].copy(),
                             how = "inner", on = "customer_unique_id")
            
            sankey_chart.loc[index,"value"] = merge.shape[0]
            sankey_chart.loc[index,"source"] = (i+ nb_of_clust)
            sankey_chart.loc[index,"target"] = (j+ nb_of_clust)
            index += 1
            
            merge = pd.merge(left = for_sankey3[for_sankey3["cluster"] == (i+(nb_of_clust*2))].copy(), 
                             right = for_sankey4[for_sankey4["cluster"] == (j+(nb_of_clust*2))].copy(),
                             how = "inner", on = "customer_unique_id")
            
            sankey_chart.loc[index,"value"] = merge.shape[0]
            sankey_chart.loc[index,"source"] = (i+ (nb_of_clust*2))
            sankey_chart.loc[index,"target"] = (j+ (nb_of_clust*2))
            index += 1
        
    return sankey_chart


In [ ]:
def add_link_colours(df):
    """
    Function to get the link colours

    Parameters:
    df: dataframe resulting from the make_df_for_sankey function

    Returns:
    df: dataframe with added link_colour column
    """
    
    df["link_colour"] = np.nan
    df.link_colour[(df["source"] == 0) | (df["source"] == 5)| (df["source"] == 10)] = "lightcoral"
    df.link_colour[(df["source"] == 1) |(df["source"] == 6) |(df["source"] == 11)] = "gold"
    df.link_colour[(df["source"] == 2) | (df["source"] == 7) | (df["source"] == 12)] = "plum"
    df.link_colour[(df["source"] == 3)| (df["source"] == 8) | (df["source"] == 13)] = "deepskyblue"
    df.link_colour[(df["source"] == 4) | (df["source"] == 9) |(df["source"] == 14)] = "bisque"
    return df


In [ ]:
sankey_chart_previous_fit = make_df_for_sankey(sankey_chart_previous_fit.copy(), 
                                               5, for_sankey_period_one.copy(), 
                                               for_previous_fitsankey2.copy(),
                                               for_previous_fitsankey3.copy(), 
                                               for_previous_fitsankey4.copy())

sankey_chart_previous_fit = add_link_colours(sankey_chart_previous_fit)

In [ ]:
def add_link_newfit_colours(df):
    """
    Function to get the link colours

    Parameters:
    df: dataframe resulting from the make_df_for_sankey function

    Returns:
    df: dataframe with added link_colour column
    """
    
    df["link_colour"] = np.nan
    df.link_colour[(df["source"] == 0) | (df["source"] == 8)| (df["source"] == 14)] = "lightcoral"
    df.link_colour[(df["source"] == 1) |(df["source"] == 7) |(df["source"] == 12)] = "gold"
    df.link_colour[(df["source"] == 2) | (df["source"] == 9) | (df["source"] == 13)] = "plum"
    df.link_colour[(df["source"] == 3)| (df["source"] == 6) | (df["source"] == 11)] = "deepskyblue"
    df.link_colour[(df["source"] == 4) | (df["source"] == 5) |(df["source"] == 10)] = "bisque"
    return df


In [ ]:
sankey_chart_new_fit = make_df_for_sankey(sankey_chart_newfit.copy(), 
                                          5, for_sankey_period_one.copy(), 
                                          for_newfit_sankey2.copy(),
                                          for_newfit_sankey3.copy(), 
                                          for_newfit_sankey4.copy())

sankey_chart_new_fit = add_link_newfit_colours(sankey_chart_new_fit)

In [ ]:
for_previous_fitsankey2.value_counts()

In [ ]:
import plotly.graph_objects as go

def plot_my_sankey(sankey_chart):
    """
    Function to plot the sankey_chart, for the version with the predicted labels from the KMeans on 
    the first timeframe

    Parameters:
    sankey_chart : df resulting from the make_df_for_sankey function

    Returns:
    sankey figure

    """
    fig = go.Figure(data=[go.Sankey(
        arrangement = 'fixed',
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(color = "black", width = 0.5),
          label = sankey_chart['label'].dropna(axis=0, how='any'),
        color = sankey_chart['colour'].dropna(axis = 0, how = "any"),
            x = [1e-09, 1e-09, 1e-09, 1e-09, 1e-09, 
                 0.33, 0.33, 0.33, 0.33, 0.33,
                 0.60, 0.60, 0.60, 0.60, 0.60, 
                 1, 1, 1, 1, 1],
            y = [0.75, 0.25, 0.14, 0.60, 0.42,
                 0.85, 0.22, 0.08, 0.65, 0.44,
                 0.95, 0.20, 0.04, 0.70, 0.45,
                 1, 0.15, 0.02, 0.74, 0.42]
        ),
        link = dict(
          source = sankey_chart['source'],
          target = sankey_chart['target'],
          value = sankey_chart['value'],
        color = sankey_chart["link_colour"]))])
    
    fig.update_layout(title_text="Sankey Diagram", font_size=10,
                     autosize = False,
                      width = 900,
                      height = 600)
    
    fig.show()
    

In [ ]:
plot_my_sankey(sankey_chart_previous_fit)

In [ ]:
def plot_my_sankey_re(sankey_chart):
    """
    Function to plot the sankey_chart, for the refitted K Means models

    Parameters:
    sankey_chart : df resulting from the make_df_for_sankey function

    Returns:
    sankey figure

    """
    fig = go.Figure(data=[go.Sankey(
        arrangement = 'fixed',
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(color = "black", width = 0.5),
          label = sankey_chart['label'].dropna(axis=0, how='any'),
        color = sankey_chart['colour'].dropna(axis = 0, how = "any"),
            x = [1e-09, 1e-09, 1e-09, 1e-09, 1e-09, 
                 0.33, 0.33, 0.33, 0.33, 0.33,
                 0.60, 0.60, 0.60, 0.60, 0.60, 
                 1, 1, 1, 1, 1],
            y = [0.75, 0.25, 0.14, 0.60, 0.42,
                 0.44, 0.65, 0.22, 0.85, 0.08,
                 0.45, 0.70, 0.20, 0.04, 0.95,
                 0.42, 0.74, 1, 0.15, 0.02]
        ),
        link = dict(
          source = sankey_chart['source'],
          target = sankey_chart['target'],
          value = sankey_chart['value'],
        color = sankey_chart["link_colour"]))])
    
    fig.update_layout(title_text="Sankey Diagram", font_size=10,
                     autosize = False,
                      width = 900,
                      height = 600)
    
    fig.show()


In [ ]:
plot_my_sankey_re(sankey_chart_new_fit)